<a href="https://colab.research.google.com/github/benjaminbrown038/torch/blob/main/torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.models import resnet50, alexnet, inception_v3